# Standard Suite (v1) Metrics -- Usage Examples

This notebook will demonstrate how to call the specific functions defined in the 
{doc}`/evaluation/Metrics_StdSuite_v1`
notebook, using a small demonstration dataset.

In [ ]:
import pandas as pd
import numpy as np

## Sample Data

In [ ]:
sampleData = pd.read_csv(r"../streamflow/NWM_Benchmark_SampleData.csv", index_col='date').dropna()
print(len(sampleData.index), " Records")
sampleData.head()

## Import Metric Functions
The functions are defined in an 
{doc}`/evaluation/Metrics_StdSuite_v1`.  
They are imported for use here by running that notebook from within the following cell:

In [ ]:
%run ../../Metrics_StdSuite_v1.ipynb
# This brings functions defined in external notebook into this notebook's namespace.

The functions are now available here, to run against our sample data.  These are called with two
arguments: an array/series of observed values and an array/series of modeled/simulated values.

A couple of examples: 

In [ ]:
# Mean Square Error
MSE(obs=sampleData['obs'], sim=sampleData['nwm'])

In [ ]:
# Kling-Gupta efficiency
KGE(obs=sampleData['obs'], sim=sampleData['nwm'])

## Create Composite Benchmark
It is useful to combine several of these metrics into a single benchmark routine, which returns a 
pandas [Series](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html) of 
the assembled metrics.

This example computes those metrics which might apply to the `streamflow` variable. 

In [ ]:
def compute_benchmark(df):
    obs = df['obs']
    sim = df['nwm']
    return pd.Series(
        data={
            'NSE': NSE(obs, sim),
            'KGE': KGE(obs, sim),
            'logNSE': logNSE(obs, sim),
            'pbias': pbias(obs, sim),
            'rSD': rSD(obs, sim),
            'pearson': pearson_r(obs, sim),
            'spearman': spearman_r(obs, sim), 
            'pBiasFMS': pBiasFMS(obs, sim),
            'pBiasFLV': pBiasFLV(obs, sim),
            'pBiasFHV': pBiasFHV(obs, sim)
        },
        name=df['site_no'].iloc[0], # special case -- 'site_no' column
        dtype='float64'
    )

In [ ]:
compute_benchmark(sampleData)

## Streamflow and FDC plots
In the case of `streamflow`, the NWM standard suite offers a way to plot the Flow Duration Curve when calculating the pBias metrics per Yilmaz et al. This mechanism uses [matplotlib](https://matplotlib.org/) to implement the figures. 

Some examples: 

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 1, figsize=(6, 3), dpi=150)
ax = FDCplot(sampleData['obs'], sampleData['nwm'], ax, segment='mid')
ax.set_title("Gage 1104200 - Mid")
plt.show()

In [ ]:
# Same fig, but with "segment='lo'"
fig, ax = plt.subplots(1, 1, figsize=(6, 3), dpi=150)
ax = FDCplot(sampleData['obs'], sampleData['nwm'], ax, segment='lo')
ax.set_title("Gage 1104200 - Low")
plt.show()

In [ ]:
# Same fig, but with "segment='hi'"
fig, ax = plt.subplots(1, 1, figsize=(6, 3), dpi=150)
ax = FDCplot(sampleData['obs'], sampleData['nwm'], ax, segment='hi')
ax.set_title("Gage 1104200 - High")
plt.show()

The default behavior is to plot the Y axis log-scale, and to leave units off of the flow rate.
If you would like to manipulate these parameters, you may adjust the `ax` after calling `FDCplot()`
(see example, next cell). In general, any of the matplotlib parameters can be adjusted after
`FDCplot()` in order to customize the figure.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 3), dpi=150)
ax = FDCplot(sampleData['obs'], sampleData['nwm'], ax, segment='mid')
ax.set_yscale('linear')
ax.set_ylabel("Flow Rate $m^3 / s$") # << labels can contain LaTex-style math between $ chars
plt.show()